In [1]:
import pandas as pd, numpy as np
from sklearn.preprocessing import Normalizer
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df_1 = pd.read_csv('base_one.csv')

In [3]:
df_2 = pd.read_csv('vtor_case2.csv', encoding='PT154', sep=';')

In [4]:
df_1 = df_1.rename(columns={'Координата Локации':'location_coord', 
                            'ID территории': 'ID_territory', 
                            'Адрес':'addr', 
                            'Площадь Локации':'location_square'})

In [5]:
df_2 = df_2.rename(columns=({'Долгота':'longitude', 'Широта':'altitude',
                     'Этаж':'floor', 'Тип_квартиры':'flat_type', 
                     'Год_постройки':'build_year', 'Материал_дома':'house material',
                     'Тип_дома':'house_type', 'Кол-во_комнат':'rooms', 'Улица':'street'}))
df_2.head(2)

,longitude,altitude,floor,flat_type,build_year,house material,house_type,rooms,street
0,82.93057372883538,55.036120884945944,11/16,улучшенной планировки,2012,кирпич,спецпроект,3.0,Романова
1,82.893498,55.127615,5/5,улучшенной планировки,1996,панель,-,1.0,Кубовая


In [6]:
df_1.head(2)

,location_coord,ID_territory,addr,location_square
0,"[55.04659173650061, 82.88206745502762]",5,"обл. Новосибирская, г. Новосибирск, ул. Дуси К...",4208.0
1,"[55.04764904059164, 82.86751319212686]",5,"Новосибирская обл, г Новосибирск, ул Сухарная",117.0


In [7]:
df_2.head(2)

,longitude,altitude,floor,flat_type,build_year,house material,house_type,rooms,street
0,82.93057372883538,55.036120884945944,11/16,улучшенной планировки,2012,кирпич,спецпроект,3.0,Романова
1,82.893498,55.127615,5/5,улучшенной планировки,1996,панель,-,1.0,Кубовая


Цель тестового задания:
1. Найти в Базе Локаций участки на которых расположены жилые многоквартирные дома.
 - Создать столбцы в Базе Локаций 
  "Жилой дом" - да, нет, возможно, неопределенно.  
  "Этажность дома" - количество этажей в доме.  
  "Количество предложений о продаже" - в штуках.  
 - Если однозначно определить наличие жилого дома не получиться, то посчитать вероятность нахождения там дома и вывести в отдельный столбец

# Найти в Базе Локаций участки на которых расположены жилые многоквартирные дома.

Присвоим сначала всем объектам районы в которых они находятся

In [8]:
df_2 = df_2[df_2['longitude'] != '-']

In [9]:
df_1['Altitude'] = df_1['location_coord'].apply(lambda x: x.split(',')[0][1:])
df_1['Longitude'] = df_1['location_coord'].apply(lambda x: x.split(',')[1][:-2])

In [10]:
df_1['Longitude'] = df_1['Longitude'].astype('float32')
df_1['Altitude'] = df_1['Altitude'].astype('float32')
df_2['longitude'] = df_2['longitude'].astype('float32')
df_2['altitude'] = df_2['altitude'].astype('float32')

In [11]:
def distance_for_df(row, df):
    temp_df = df.copy()
    temp_df['distance'] = np.sqrt((temp_df['Longitude'] - row['longitude'])**2 + (temp_df['Altitude'] - row['altitude'])**2)
    temp_df['in_loc_square'] = temp_df['location_square'] - temp_df['distance']
    if temp_df[temp_df['in_loc_square'] > 0].shape[0] > 0:
        temp_df = temp_df[temp_df['in_loc_square'] > 0]
        temp_df = temp_df.sort_values(['distance'])
        res = temp_df.iloc[0]['addr']
    else:
        temp_df = temp_df.sort_values(['distance'])
        res = temp_df.iloc[0]['addr']        
    return res

In [12]:
%%time
df_2['district'] = df_2.parallel_apply(lambda row: distance_for_df(row, df_1), axis=1)

CPU times: user 48.4 ms, sys: 37.2 ms, total: 85.5 ms
Wall time: 20.7 s


In [13]:
df_2.head(2)

,longitude,altitude,floor,flat_type,build_year,house material,house_type,rooms,street,district
0,82.930573,55.036121,11/16,улучшенной планировки,2012,кирпич,спецпроект,3.0,Романова,"Новосибирская область, г Новосибирск, ул Пугач..."
1,82.893501,55.127617,5/5,улучшенной планировки,1996,панель,-,1.0,Кубовая,"обл. Новосибирская, г. Новосибирск, ул. Победы..."


 - Создать столбцы в Базе Локаций 
  "Жилой дом" - да, нет, возможно, неопределенно.  
  "Этажность дома" - количество этажей в доме.  
  "Количество предложений о продаже" - в штуках. 

In [14]:
df_2_grouped =df_2.groupby('district').agg({'altitude':'count', 'floor':'unique'})
df_2_grouped.rename(columns={'altitude':'qty_sale_offers'}, inplace=True)
df_2_grouped.head(2)

,qty_sale_offers,floor
district,,
"Новосибирская обл, г Новосибирск",7,"[7/25, 25/25, 20/26, 10/26, 4/27, 2/26, 23/25]"
"Новосибирская обл, г Новосибирск, р-н Калининский, ул Богдана Хмельницкого, на земельном участке расположен многоэтажный жилой дом с помещениями общественного назначения, автостоянками и трансформаторной подстанцией,11/3",12,"[16/20, 10/20, 12/20, 20/20, 14/20, 8/20, 6/20..."


По хорошему надо сначала удалить дубликаты в базе локаций, но в задании этого нет и возможно это для чего-то да надо

In [15]:
res_df = df_1.merge(df_2_grouped, how='left', left_on='addr', right_on='district')
res_df.head(2)

,location_coord,ID_territory,addr,location_square,Altitude,Longitude,qty_sale_offers,floor
0,"[55.04659173650061, 82.88206745502762]",5,"обл. Новосибирская, г. Новосибирск, ул. Дуси К...",4208.0,55.046593,82.882065,1.0,"[nan, 1/4]"
1,"[55.04764904059164, 82.86751319212686]",5,"Новосибирская обл, г Новосибирск, ул Сухарная",117.0,55.047649,82.867516,139.0,"[18/25, 24/24, 9/14, 4/5, 5/7, 24/25, 16/24, 7..."


In [16]:
res_df['dwelling_house'] = res_df['qty_sale_offers'].parallel_apply(lambda x: 'yes' if x > 0 else 'no')

In [17]:
res_df

,location_coord,ID_territory,addr,location_square,Altitude,Longitude,qty_sale_offers,floor,dwelling_house
0,"[55.04659173650061, 82.88206745502762]",5,"обл. Новосибирская, г. Новосибирск, ул. Дуси К...",4208.0,55.046593,82.882065,1.0,"[nan, 1/4]",yes
1,"[55.04764904059164, 82.86751319212686]",5,"Новосибирская обл, г Новосибирск, ул Сухарная",117.0,55.047649,82.867516,139.0,"[18/25, 24/24, 9/14, 4/5, 5/7, 24/25, 16/24, 7...",yes
2,"[55.05070809130114, 82.8672947022178]",5,NaN,52.0,55.050709,82.867294,NaN,NaN,no
3,"[55.04998700650803, 82.87280630356447]",5,"обл. Новосибирская, г. Новосибирск, ул. Тимиря...",50701.0,55.049988,82.872803,NaN,NaN,no
4,"[55.053754353860235, 82.8843948900113]",5,"обл. Новосибирская, г. Новосибирск, ул. Тимиря...",414.0,55.053753,82.884392,NaN,NaN,no
...,...,...,...,...,...,...,...,...,...
1487,"[55.07405313136259, 82.9414407859326]",8,"Новосибирская обл., г. Новосибирск, ул. Новая",10401.0,55.074055,82.941444,NaN,NaN,no
1488,"[55.066923747238505, 82.92790165867125]",7,"обл. Новосибирская, г. Новосибирск, ул. Залесс...",8.0,55.066925,82.927902,6.0,"[3/25, 24/26, 22/26, 22/25, 6/25, 2/25]",yes
1489,"[55.066923747238505, 82.92790165867125]",8,"обл. Новосибирская, г. Новосибирск, ул. Залесс...",8.0,55.066925,82.927902,6.0,"[3/25, 24/26, 22/26, 22/25, 6/25, 2/25]",yes
1490,"[55.064025527570685, 82.93329162797811]",7,"Новосибирская область, г Новосибирск, ул Танковая",6592.0,55.064026,82.933289,337.0,"[10/20, 15/20, 6/20, 17/20, 13/18, 8/18, 14/25...",yes


### Если однозначно определить наличие жилого дома не получиться, то посчитать вероятность нахождения там дома и вывести в отдельный столбец

идея в следующем - мы сгруппируем районы, в которых есть жилые дома в кластеры и посмотрим, насколько далеко район, в котором надо определить вероятность нахождения жилья, находится далеко от центра кластера. 

In [18]:
from sklearn.cluster import KMeans

In [19]:
dwelling_house = res_df[res_df['dwelling_house'] == 'yes'][['Altitude', 'Longitude']].to_numpy()

Пусть будет 5 кластеров, это можно в любой момент изменить, надо только отрисовать и посмотреть глазами на распределение

In [20]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(dwelling_house)

In [21]:
train = res_df[res_df['dwelling_house'] == 'yes']

In [22]:
train['cluster'] = kmeans.labels_
train.head()

<ipython-input-22-1d062c2d3b3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['cluster'] = kmeans.labels_


,location_coord,ID_territory,addr,location_square,Altitude,Longitude,qty_sale_offers,floor,dwelling_house,cluster
0,"[55.04659173650061, 82.88206745502762]",5,"обл. Новосибирская, г. Новосибирск, ул. Дуси К...",4208.0,55.046593,82.882065,1.0,"[nan, 1/4]",yes,1
1,"[55.04764904059164, 82.86751319212686]",5,"Новосибирская обл, г Новосибирск, ул Сухарная",117.0,55.047649,82.867516,139.0,"[18/25, 24/24, 9/14, 4/5, 5/7, 24/25, 16/24, 7...",yes,1
8,"[55.04660124294934, 82.8716448237154]",5,"обл. Новосибирская, г. Новосибирск, ул. Сухарн...",2548.0,55.046600,82.871643,409.0,"[6/9, 4/5, 2/11, 4/4, 1/5, 2/5, 6/12, 5/7, 5/1...",yes,1
11,"[55.04590863378286, 82.88197371032018]",5,"обл. Новосибирская, г. Новосибирск, ул. Дуси К...",4741.0,55.045910,82.881973,3.0,"[8/9, 9/9]",yes,1
14,"[55.05078063384141, 82.88916324756984]",5,"обл. Новосибирская, г. Новосибирск, ул. Дуси К...",6543.0,55.050781,82.889160,127.0,"[2/9, 8/10, 9/10, 4/9, 10/10, 3/5, 14/17, 2/16...",yes,1


Найдем среднее расстояние от центра района до центра кластера

In [23]:
def distance_to_center(row, centroids):
    cords_centroid = centroids[row['cluster']]
    dist = np.sqrt((row['Longitude'] - cords_centroid[1])**2 + (row['Altitude'] - cords_centroid[0])**2)
    return dist

In [24]:
train['distance_to_centriod'] = train.parallel_apply(lambda row: distance_to_center(row, kmeans.cluster_centers_), axis=1)

<ipython-input-24-d7292898bcd0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['distance_to_centriod'] = train.parallel_apply(lambda row: distance_to_center(row, kmeans.cluster_centers_), axis=1)


Теперь найдем к какому кластеру принадлежит каждый из оставшихся районов

In [25]:
other_buildings = res_df[res_df['dwelling_house'] != 'yes']
other_buildings.head(2)

,location_coord,ID_territory,addr,location_square,Altitude,Longitude,qty_sale_offers,floor,dwelling_house
2,"[55.05070809130114, 82.8672947022178]",5,NaN,52.0,55.050709,82.867294,NaN,NaN,no
3,"[55.04998700650803, 82.87280630356447]",5,"обл. Новосибирская, г. Новосибирск, ул. Тимиря...",50701.0,55.049988,82.872803,NaN,NaN,no


In [26]:
other_buildings['cluster'] = kmeans.predict(other_buildings[['Altitude', 'Longitude']].to_numpy())

<ipython-input-26-f5efab50b531>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_buildings['cluster'] = kmeans.predict(other_buildings[['Altitude', 'Longitude']].to_numpy())


Найдем расстояние до центра каждого кластера

In [27]:
other_buildings['distance_to_centriod'] = other_buildings.parallel_apply(lambda row: distance_to_center(row, kmeans.cluster_centers_), axis=1)

<ipython-input-27-618460015048>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_buildings['distance_to_centriod'] = other_buildings.parallel_apply(lambda row: distance_to_center(row, kmeans.cluster_centers_), axis=1)


Так как нам надо получить нечто похожее на вероятность, то загоним все расстояния в диапазон от 0 до 1, где 0 - это максимально удаленный район, а 1 это максимально прибилженный к центру кластера

In [28]:
def norm(row, df):
    tmp_df = df[df['cluster'] == row['cluster']]
    normalized = (row['distance_to_centriod']-min(tmp_df['distance_to_centriod']))/(max(tmp_df['distance_to_centriod'])-min(tmp_df['distance_to_centriod']))
    return normalized

In [29]:
%%time
other_buildings['norm_distance_to_centriod'] = other_buildings.parallel_apply(lambda row: norm(row, other_buildings) * 100, axis=1)

CPU times: user 1.23 s, sys: 44.4 ms, total: 1.28 s
Wall time: 346 ms


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Теперь сведём это всё вместе

In [30]:
other_buildings

,location_coord,ID_territory,addr,location_square,Altitude,Longitude,qty_sale_offers,floor,dwelling_house,cluster,distance_to_centriod,norm_distance_to_centriod
2,"[55.05070809130114, 82.8672947022178]",5,NaN,52.0,55.050709,82.867294,NaN,NaN,no,1,0.011810,37.962997
3,"[55.04998700650803, 82.87280630356447]",5,"обл. Новосибирская, г. Новосибирск, ул. Тимиря...",50701.0,55.049988,82.872803,NaN,NaN,no,1,0.006283,19.537162
4,"[55.053754353860235, 82.8843948900113]",5,"обл. Новосибирская, г. Новосибирск, ул. Тимиря...",414.0,55.053753,82.884392,NaN,NaN,no,1,0.007753,24.436710
5,"[55.05060111427259, 82.88639193403974]",5,"обл. Новосибирская, г. Новосибирск, ул. Союза ...",7183.0,55.050602,82.886391,NaN,NaN,no,1,0.007868,24.821812
6,"[55.05242373372826, 82.879226359625]",5,"обл. Новосибирская, г. Новосибирск, ул. Гипсов...",815.0,55.052425,82.879227,NaN,NaN,no,1,0.004121,12.329725
...,...,...,...,...,...,...,...,...,...,...,...,...
1483,"[55.0760050648901, 82.93533059878585]",8,"обл. Новосибирская, г. Новосибирск, ул. Магада...",4409.0,55.076004,82.935333,NaN,NaN,no,2,0.010481,32.946783
1484,"[55.07185650788018, 82.92253626553104]",7,"обл. Новосибирская, г. Новосибирск, ул. Каунас...",2068.0,55.071857,82.922539,NaN,NaN,no,2,0.005094,13.760968
1485,"[55.07185650788018, 82.92253626553104]",8,"обл. Новосибирская, г. Новосибирск, ул. Каунас...",2068.0,55.071857,82.922539,NaN,NaN,no,2,0.005094,13.760968
1486,"[55.07405313136259, 82.9414407859326]",7,"Новосибирская обл., г. Новосибирск, ул. Новая",10401.0,55.074055,82.941444,NaN,NaN,no,2,0.015057,49.244310


In [31]:
df_1['dwelling_house'] = res_df['dwelling_house']
df_1['floor'] = res_df['floor']
df_1['qty_sale_offers'] = res_df['qty_sale_offers']
df_1['probability'] = 100
df_1.loc[other_buildings.index, 'probability'] = other_buildings['norm_distance_to_centriod']
df_1['qty_sale_offers'] = df_1['qty_sale_offers'].fillna(0)

In [32]:
df_1

,location_coord,ID_territory,addr,location_square,Altitude,Longitude,dwelling_house,floor,qty_sale_offers,probability
0,"[55.04659173650061, 82.88206745502762]",5,"обл. Новосибирская, г. Новосибирск, ул. Дуси К...",4208.0,55.046593,82.882065,yes,"[nan, 1/4]",1.0,100.000000
1,"[55.04764904059164, 82.86751319212686]",5,"Новосибирская обл, г Новосибирск, ул Сухарная",117.0,55.047649,82.867516,yes,"[18/25, 24/24, 9/14, 4/5, 5/7, 24/25, 16/24, 7...",139.0,100.000000
2,"[55.05070809130114, 82.8672947022178]",5,NaN,52.0,55.050709,82.867294,no,NaN,0.0,37.962997
3,"[55.04998700650803, 82.87280630356447]",5,"обл. Новосибирская, г. Новосибирск, ул. Тимиря...",50701.0,55.049988,82.872803,no,NaN,0.0,19.537162
4,"[55.053754353860235, 82.8843948900113]",5,"обл. Новосибирская, г. Новосибирск, ул. Тимиря...",414.0,55.053753,82.884392,no,NaN,0.0,24.436710
...,...,...,...,...,...,...,...,...,...,...
1487,"[55.07405313136259, 82.9414407859326]",8,"Новосибирская обл., г. Новосибирск, ул. Новая",10401.0,55.074055,82.941444,no,NaN,0.0,49.244310
1488,"[55.066923747238505, 82.92790165867125]",7,"обл. Новосибирская, г. Новосибирск, ул. Залесс...",8.0,55.066925,82.927902,yes,"[3/25, 24/26, 22/26, 22/25, 6/25, 2/25]",6.0,100.000000
1489,"[55.066923747238505, 82.92790165867125]",8,"обл. Новосибирская, г. Новосибирск, ул. Залесс...",8.0,55.066925,82.927902,yes,"[3/25, 24/26, 22/26, 22/25, 6/25, 2/25]",6.0,100.000000
1490,"[55.064025527570685, 82.93329162797811]",7,"Новосибирская область, г Новосибирск, ул Танковая",6592.0,55.064026,82.933289,yes,"[10/20, 15/20, 6/20, 17/20, 13/18, 8/18, 14/25...",337.0,100.000000
